# Setup
Run the cell below to import and install the dependencies needed for the project.

In [ ]:
# Functions to use along widgets
##General Imports
from IPython.display import HTML, display, Markdown, Video, clear_output
from ipywidgets import Layout
# Faciliate file selection
from tkinter import *
from tkinter import filedialog
# Widget Packages
import ipywidgets as widgets
# jupyter nbextension enable --py widgetsnbextension
# Used for local directory
import os
import sys
# import TSU.json_util as json_util
# from TSU.json_util import test_train_json

from IPython.display import HTML
from util_modules.ui_util.progress_bar import progress_bar

##Imports for captioning
import pandas
import cv2
import re

##Imports for Feature Extraction
from os import walk
from video_features.utils.utils import build_cfg_path
from omegaconf import OmegaConf
import torch
from video_features.features_models.i3d.extract_i3d import ExtractI3D

##Imports for TSU
from util_modules.json_util.test_train_json import inference_json

##Imports for Second Algorithm (STEP/3D3)

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Show/hide code"></form>''')

# Welcome to the HOI Interactive Notebook Tool 

### Quick-Start:
- Refer to the instructions and documentation provided in the Markdown cells of each of the sections below to better understand the functions of each feature.

### Pre-Requisites:
- 

## 1. Initialisation
Check if your device has a [CUDA-supported GPU](https://developer.nvidia.com/cuda-gpus#compute).

In [13]:
# layout init.
btn_layout = widgets.Layout(width='45%', height='40px')
btn_sm_layout = widgets.Layout(width='20%', height='40px') 
label_layout = Layout(width='200px',height='auto')
ddl_layout = widgets.Layout(width='45%', height= 'auto')


# Widget
btn_check_if_cuda = widgets.Button(description="Check if your device is CUDA supported",
    layout = btn_layout, button_style='info')

cuda_output = widgets.Output()

def check_if_cuda(b):
    with cuda_output:
        cuda_output.clear_output()
        if (torch.cuda.is_available()):
            print('Your computer is CUDA supported.')
        else:
            print('Your computer is not CUDA supported.')
    # --- write script here to check if user's machine if cuda supported
    return True

btn_check_if_cuda.on_click(check_if_cuda)

# Display
display(btn_check_if_cuda, cuda_output)

Button(button_style='info', description='Check if your device is CUDA supported', layout=Layout(height='40px',…

Output()

## 2. Select Pipeline
The following Machine Learning (ML) pipelines have been integrated with this project to allow you to have a hand at Activity Detection with ease. Click on any of the following links to learn more about their source repositiories if you're interested.
- [Toyota Smarthome (TSU)](https://project.inria.fr/toyotasmarthome/)
- [NVIDIA STEP)](https://github.com/NVlabs/STEP/)

In [14]:
# Pre-requisites
"""
List of Machine Learning stuff
Consider making it a text file to store/edit?
"""
ml_list = ['Toyota Smart Home', 'Nvidia STEP']

# Widgets
btn_select_pipeline = widgets.Button(description="Confirm",
    layout = btn_layout, button_style='info')

ddl_pipeline = widgets.Dropdown(
    options=ml_list,
    value=ml_list[0],layout = btn_layout,
    description='Select ML Pipeline:', style={'description_width': 'initial'})

# Output
widgetset = widgets.Output()

# Function
def selectWidgetSet(b):
    with widgetset:
        widgetset.clear_output()
        print(ddl_pipeline.value + " is selected as the pipeline.")

btn_select_pipeline.on_click(selectWidgetSet)

# Display
tooltip = widgets.Label("Supported Video Types: MP4, WebM, and OGG.")
mlBox = widgets.VBox([ddl_pipeline,btn_select_pipeline,widgetset])
mlBox

## 3. Data Exploration
Select and view playback of any video from the `data` folder. 

In [15]:
# Logic
def populateList(fileDirectory, fileType):
    folder_files = os.listdir(fileDirectory) #You can also use full path.
    print("`data/rgbVideos` directory contains {len_folder} file(s).".format(len_folder=len(folder_files)))
    fileList = []
    for file in folder_files:
        if file.endswith(fileType):
            fileList.append(file)
    return fileList

fileList = populateList('data/rgbVideos', ".mp4")
        
# Widget
label_data_files= widgets.Label("Select Video(s):", layout = label_layout)
ddl_data_selected = widgets.Dropdown(
    options = fileList,
    value= fileList[0],
    layout = ddl_layout)

btn_video = widgets.Button(
    description='Play video', layout = btn_sm_layout, button_style='info'
)

btn_clear = widgets.Button(
    description='Clear Playback', layout = btn_sm_layout
)

# Display output
video_output = widgets.Output()
def explore_video(b):
    with video_output:
        # Create Video File
        video_output.clear_output()
#         html_video_code = '<video width="80%" height="80%" controls><source src="./data/rgbVideos/{fileName}" type="video/mp4"></video>'.format(fileName = ddl_data_selected.value)
        html_video_code = '<video width="80%" height="80%" controls><source src="./videosCaptionOutput/P02T02C06.mp4" type="video/mp4"></video>'

        video_output.append_display_data(HTML(html_video_code))

btn_video.on_click(explore_video)

# CLear output
def clear_video(b):
    with video_output:
        video_output.clear_output()
        
btn_clear.on_click(clear_video)

# Display
data_box = widgets.VBox([widgets.HBox([label_data_files, ddl_data_selected])])
data_btn_box = widgets.VBox([widgets.HBox([btn_clear, btn_video]),video_output])

display(data_box)
display(data_btn_box)



`data/rgbVideos` directory contains 536 file(s).


## 4. Feature Extraction
This section performs feature extractions from videos into numpy format, so that it can be used to perform inferencing, training and testing.

### Parameters:
- Folder & File
    - Locate the directory where your videos are stored for feature extraction
    - Select one one or more videos
- Stream Type
    - Select between `null, rgb, flow` stream types

In [ ]:
# Widgets

sub_folders = [name for name in os.listdir('./data') if os.path.isdir(os.path.join('./data', name))]
sub_folders.remove('.ipynb_checkpoints')
sub_folders = sorted(sub_folders,reverse=True)

list_of_videos = []

label_video_folder = widgets.Label("Select Folder:", layout = label_layout)
ddl_video_folder = widgets.Dropdown(
    options=sub_folders,
    layout = btn_layout)

label_stream = widgets.Label("Select Stream:", layout = label_layout)
ddl_stream = widgets.Select(
    options=['null', 'rgb', 'flow'],
    value='null',
    layout = ddl_layout)

btn_confirm_folder = widgets.Button(description="Confirm Folder", layout = btn_layout, button_style='info')

def selectvideos(b):
    for (dirpath, dirnames, filenames) in walk('./data/rgbVideos/'):
        list_of_videos.extend(filenames)
        break
    label_feature_files= widgets.Label("Select Video(s):", layout = label_layout)
    ddl_feature_files = widgets.SelectMultiple(
        options=list_of_videos,
        layout = ddl_layout)
    test_box = widgets.VBox([widgets.HBox([label_feature_files, ddl_feature_files]),
                     widgets.HBox([label_stream, ddl_stream]), btn_extract])
    display(test_box)
    
btn_confirm_folder.on_click(selectvideos)

btn_extract = widgets.Button(description="Extract Features", layout = btn_layout, button_style='info')

# Display
feature_box = widgets.VBox([widgets.HBox([label_video_folder, ddl_video_folder]),
                            btn_confirm_folder])
feature_box
    

## 5. Inference
This is the inference section where users can test and evaluate how accurate their model is in the actual video itself, comparing the ground truth and generated annotations.

### Parameters:
- Model 
  - Select from one of the pre-trained ML models
- Input Video 
  - Select an input video from the `/data/rgbVideos` directory

In [16]:
# Logic
model_dir = './TSU/models/'
# listdir() returns a list containing the names of the entries in the directory given by path.
modelList = os.listdir("./TSU/models")
train_modelList = os.listdir("./TSU/PDAN")
modelList += train_modelList

## retrieve all TSU videos to dropdown list
folder_files = os.listdir('data/rgbVideos') 
print("data/rgbVideos` directory contains {len_folder} file(s).".format(len_folder=len(folder_files)))
fileList=[]
for file in folder_files:
    fileList.append(file)

# Widgets
label_inference_model = widgets.Label("Select Model:", layout = label_layout)
ddl_inference_model = widgets.Dropdown(
    options = modelList,
    value = modelList[0],
    layout = btn_layout)

label_inference_video = widgets.Label("Select Input Video:", layout = label_layout)
selected_input_video = widgets.Dropdown(
    options = fileList,
    value= fileList[0],
    layout = btn_layout)

btn_inference = widgets.Button(description="Run Inferencing", layout = btn_layout, button_style='info')

# Display
model_box = widgets.HBox([label_inference_model, ddl_inference_model])
inference_box = widgets.VBox([model_box, widgets.HBox([label_inference_video, selected_input_video])])
inference_output = widgets.Output()
display(inference_box)
display(btn_inference)
display(inference_output)

###### Generate and Save captions Function ###################
class Captioning:
    def __init__(self, annotation_file_path, truth_file_path, video_file):
        self.sf = pandas.read_csv(annotation_file_path, header=0, names=['action','start','end','accuracy'], usecols=['action','start','end','accuracy'])[['action','start','end','accuracy']]
        self.df = pandas.read_csv(truth_file_path)
        self.video_path = video_file
        self.time = 0
        self.nextTime = 0
        self.nextActionTime = 0
        self.eventCounter = 0
        self.actionCounter = 0
        self.prediction = 0.0
        self.ground_truth = ""
        self.caption = ""
        self.output_path = "./videosCaptionOutput/"

    def getGroundTruth(self):
        if self.time == 0:
            self.nextTime = self.df['start_frame'].iloc[1]

        if self.time >= self.nextTime:
            try:
                self.ground_truth = self.df['event'].iloc[self.eventCounter]
                self.ground_truth =  re.sub('[_,.]', ' ', self.ground_truth)
                self.nextTime = self.df['start_frame'].iloc[self.eventCounter+1]
                self.eventCounter += 1
            except:
                pass
            
            
    def getCaption(self):
        if self.actionCounter == 0:
            self.actionCounter, = self.sf.index[self.sf['end'] == 'end']
            
        if self.time >= self.nextActionTime:
            self.actionCounter += 1
            try: 
                self.nextActionTime = float(self.sf['end'].iloc[self.actionCounter])
                self.caption = self.sf['action'].iloc[self.actionCounter]
                self.caption =  re.sub('[_,.]', ' ', self.caption)
                self.prediction = float(self.sf['accuracy'].iloc[self.actionCounter])
            except:
                pass


    def saveVideo(self):
        if not os.path.exists('./videosCaptionOutput/'):
            os.makedirs('./videosCaptionOutput/')
        cap = cv2.VideoCapture(self.video_path)
        # and our buffer to write frames
#         fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        fourcc = cv2.VideoWriter_fourcc(*'H264')
        out = cv2.VideoWriter(self.output_path+str(self.video_path[len(self.video_path)-13:len(self.video_path)-4])+'.mp4', fourcc, 25, (int(cap.get(3)),int(cap.get(4))))
        if (cap.isOpened() == False):
            print("Error opening video stream or file")
        counter = 0
        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        progress_bar_instance = progress_bar(length)
        progress_bar_instance.display_bar()
        while (cap.isOpened()):
            ret, frame = cap.read()
            if not ret:
                print(f'Video annotation is process complete.')
                break

            height, width, channels = frame.shape

            self.time = int(cap.get(cv2.CAP_PROP_POS_MSEC)/40)
            self.getGroundTruth()
            self.getCaption()
    
#             print(self.time, self.caption)
            cv2.rectangle(frame, (int(width * 0.05), int(height * 0.8)), (int(width * 0.95), int(height*0.95)), (159,159,159), -1)
            cv2.putText(frame, "Ground truth", (int(width*0.1),int(height*0.85)), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,0), 2)
            cv2.putText(frame, self.ground_truth, (int(width*0.3),int(height*0.85)), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,0), 2)
            cv2.putText(frame, "Prediction", (int(width*0.1),int(height*0.9)), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,0), 2)
            cv2.putText(frame, self.caption, (int(width*0.3),int(height*0.9)), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,0), 2)
            cv2.putText(frame, "Accuracy", (int(width*0.65),int(height*0.9)), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,0), 2)
            cv2.putText(frame, str(self.prediction), (int(width*0.8),int(height*0.9)), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,0), 2)

            #write our frame
            out.write(frame)
            cv2.imshow('frame',frame)

            key = cv2.waitKey(1)
            # define the key to
            # close the window
            if key == 'q' or key == 27:
                break
            progress_bar_instance.update_bar()

        cap.release()
        out.release()
        cv2.destroyAllWindows()       
    
# Usage Captioning(annotation_file_path, truth_file_path, video_file) 
# annotation_file_path (e.g. "./data/generatedAnnotations/PDAN_TSU_RGB_P02T01C06.csv" )
# truth_file_path (e.g. "./data/annotations/P02/P02T01C06.csv" )
# video_file (e.g. "./data/rgbVideos/P02T01C06.mp4")

###### Generate CSV After running Inferencing ###########################
def run_inferencing(b):
    with inference_output:
        clear_output()
        video_data = selected_input_video.value.replace(".mp4","")
        inference_json(video_data, "TSU")
        if (ddl_inference_model.value == "PDAN_TSU_RGB"):
            sub_dir = "./TSU/models/"
        else:
            sub_dir = "./TSU/PDAN/"
        inf_dir = sub_dir + ddl_inference_model.value
        inference_dataset = "inference_smarthome_CS_51.json"
        dataset_type = "TSU"
        %run ./TSU/inferencing.py -load_model $inf_dir -video_name $selected_input_video.value -dataset $inference_dataset -dataset_type $dataset_type
        sub_folder = selected_input_video.value[:3]
        gt_annotation_directory = "./data/annotations/{folder}/".format(folder=sub_folder)
        generated_annotation_directory = "./data/generatedAnnotations/"
        annotation_file = selected_input_video.value.replace(".mp4",".csv")
        gen_annotation_file = "{loaded_model}_{vid_name}".format(loaded_model=ddl_inference_model.value, vid_name=annotation_file)
        video_directory = "./data/rgbVideos/{vid_name}".format(vid_name=selected_input_video.value)
        print("video is processing now..")
        test1 = Captioning(generated_annotation_directory+gen_annotation_file, gt_annotation_directory+annotation_file, video_directory)
        test1.saveVideo()
        html_video_code = '<video width="80%" height="80%" controls><source src="./videosCaptionOutput/{video}" type="video/mp4"></video>'.format(video=selected_input_video.value)

        inference_output.append_display_data(HTML(html_video_code))

btn_inference.on_click(run_inferencing)



data/rgbVideos` directory contains 536 file(s).


Button(button_style='info', description='Run Inferencing', layout=Layout(height='40px', width='45%'), style=Bu…

Output()

## 5. Training
//// INSERT DESCRIPTION HERE //////

### Select dataset:
- Select a dataset from the `data` folder for input for the training & testing functionalities.

### Initialise Training
  - Video Type
  - Model Name
  - Batch_Size
  - Epoch
  - Kernel

## 6. Testing
//// INSERT DESCRIPTION HERE //////

Your dataset has already been selected in section 5.

In [ ]:
print("---print number of videos in testing here")

# Widget
btn_testing = widgets.Button(description="Run Testing", layout = btn_layout, button_style='info')

# Display
display(model_box)
display(btn_testing)